In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import json
import re
import requests
import tweepy

### Data Gathering Section

**Loading twitter archive dataset**

In [2]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

**Getting image prediction of different dogs at [Udacity server](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)**

In [ ]:
response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [ ]:
# Saving the image prediction into a tsv file
with open('image_predictions.tsv', 'wb') as file: 
    file.write(response.content)

**Loading image_predictions.tsv file into a dataframe**

In [3]:
image_pred = pd.read_csv('image_predictions.tsv', sep='\t')

**Getting more data information about different tweetids in twitter archive dataframe**

In [ ]:
from getpass import getpass # Package to get consumer key and secret as a password

In [ ]:
consumer_key = getpass("Consumer key:") # Getting the consumer key from stdin
consumer_secret = getpass("Consumer secret:") # Getting the consumer secret from stdin

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 

access_token = getpass("Access Token:")
access_secret = getpass('Access secret:')

auth.set_access_token(access_token, access_secret) # Setting the access tokens

api = tweepy.API(auth, wait_on_rate_limit=True) # Initializing the API

with open('tweet_json.txt', 'w') as file:
    for tweetid in twitter_archive['tweet_id'].values:
        try:
            tweet = api.get_status(tweetid, tweet_mode='extended') # Getting different tweets
            json.dump(tweet._json, file) # Saving the tweet as json file
            file.write('\n') # Adding newline character after saving 
        except:
            print(f'{tweetid} not found!') # This will get prompt when tweet id is not found

**Loading the tweets retrieved from twitter into a dataframe**

In [4]:
tweets_use = (pd.read_json('tweet_json.txt', lines=True)
                .loc[:, ['id', 'retweet_count', 'favorite_count']])

### Accessing Data

#### Asessing Twitter archive file

In [5]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [6]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

> Observation

- Misrepresentation of data type e.g., `tweet_id` column is integer instead of string.
- There are only 78 values in `in_reply_to_status_id` and `in_reply_to_user_id` columns.
- Timestamp column is a string, instead of datetime object.
- There are 181 non-null values in `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` columns.
- Out of 2356 rows, `expanded_urls` columns has 2297 non-null values.
- Misrepresentation of NA value in different columns.
- Dog stages has 4 columns i.e., `doggo`, `floofer`, `pupper`, `puppo`

**Number of NA in the dataset**

In [7]:
twitter_archive.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

##### Ratings column

**Descriptive analysis on numerical columns**

In [8]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


> Observation

Both minimum and maximum ratings numerator and denominator are wrong because ratings numerator or denominator are not suppose to be this low or high!

In [9]:
wrong_denom = (twitter_archive.query('rating_denominator != 10')[
    ['text', 'rating_numerator', 'rating_denominator']]
    .head())
wrong_denom

,text,rating_numerator,rating_denominator
313,@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0
342,@docmisterio account started on 11/15/15,11,15
433,The floofs have been released I repeat the flo...,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24,7
784,"RT @dog_rates: After so many requests, this is...",9,11


In [10]:
wrong_denom.loc[313, 'text']

"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"

> Observation

As this can be seen, some ratings numerator and denominator are chosen wrongly from the `text` column.

In [11]:
# Checking for wrong ratings representation
wrong_repr = (twitter_archive[twitter_archive['text']
                .str.contains(r'\d+.\d+/\d+')]
                [['text', 'rating_numerator', 'rating_denominator']])
wrong_repr.head()

,text,rating_numerator,rating_denominator
45,This is Bella. She hopes her smile made you sm...,5,10
188,@dhmontgomery We also gave snoop dogg a 420/10...,420,10
189,@s8n You tried very hard to portray this good ...,666,10
290,@markhoppus 182/10,182,10
313,@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0


In [12]:
wrong_repr.loc[45, 'text']

'This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948'

> Observation

- As can be seen, some numerators and denominators were wrongly represented. Instead 13.5/10, 5/10 was used in the dataframe.
- Text column have an unknow url at the end of each tweet.

##### Tweet id column

**Checking for duplicate values on `tweet_id`**

In [13]:
twitter_archive.tweet_id.duplicated().any()

False

> Observation

All tweets are unique, there is no duplicate

##### Name column

In [14]:
twitter_archive['name'].value_counts()

None          745
a              55
Charlie        12
Cooper         11
Lucy           11
             ... 
Dex             1
Ace             1
Tayzie          1
Grizzie         1
Christoper      1
Name: name, Length: 957, dtype: int64

In [15]:
twitter_archive['name'].value_counts().sort_index()

Abby            2
Ace             1
Acro            1
Adele           1
Aiden           1
               ..
such            1
the             8
this            1
unacceptable    1
very            5
Name: name, Length: 957, dtype: int64

> Observation

There are some unknown dog names i.e., a, "None", "such", "the", "this", "unacceptable", "very" etc., But all unknown names have the same pattern except for None (lower case).

##### Dog stages

In [16]:
# Checking for the unique values
for column in twitter_archive.loc[:, "doggo":].columns:
    print(column, "column values:", twitter_archive[column].unique())

doggo column values: ['None' 'doggo']
floofer column values: ['None' 'floofer']
pupper column values: ['None' 'pupper']
puppo column values: ['None' 'puppo']


> Observation
None is not saved as N/A

#### Assessing additional tweet information

In [17]:
tweets_use.head()

,id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [18]:
tweets_use.id.duplicated().any()

False

> Observation

All tweets are unique, there is no duplicate

In [19]:
# brief information of tweets
tweets_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


> Observation

- id is a string and not integer

**Descriptive analysis on numerical columns**

In [20]:
tweets_use.describe()

,id,retweet_count,favorite_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,3164.797366,8080.968564
std,6.852812e+16,5284.770364,11814.771334
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,624.500000,1415.000000
50%,7.194596e+17,1473.500000,3603.500000
75%,7.993058e+17,3652.000000,10122.250000
max,8.924206e+17,79515.000000,132810.000000


#### Assessing image prediction

Description of image prediction columns:

- `tweet_id`: the unique identifier for each tweet
- `jpg_url`: dog's image URL
- `img_num`: the image number that corresponded to the most confident prediction
- `p1`: algorithm's #1 prediction for the image in the tweet
- `p1_conf`: how confident the algorithm is in its #1 prediction
- `p1_dog`: whether or not the #1 prediction is a breed of dog
- `p2`: algorithm's #2 prediction for the image in the tweet
- `p2_conf`: how confident the algorithm is in its #2 prediction
- `p2_dog`: whether or not the #2 prediction is a breed of dog
- `p3`: algorithm's #3 prediction for the image in the tweet
- `p3_conf`: how confident the algorithm is in its #3 prediction
- `p3_dog`: whether or not the #3 prediction is a breed of dog

In [21]:
image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [22]:
# brief info of image prediction dataframe
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


> Observation
- Data quality issue i.e., misrepresentation of `tweet_id`.
- Also, inconsistent dog breed name i.e., uppercase and lowercase
- Names separated with "_" instead of " ".

**Descriptive analysis on numerical columns**

In [23]:
image_pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


#### Quality
**Twitter archive table**
- Erroneous data type e.g., tweet id is string instead of int and timestamp is datetime object instead of string.
- Misrepresentation of NA values in different columns such as Doggo, Floofer, Pupper, Puppo etc.
- Redundant urls in source column (Only text in \<a>text\</a> is needed)
- Unknown url at end tweet texts
- Invalid ratings representation i.e., some ratings are wrongly represented e.g., 5/10 instead of 13.5/10.
- Invalid huge ratings numerator and denominator.
- Retweeted tweets (181) and replies (78) not needed.
- Invalid dogs name e.g., "a", "such", "quite", "None", etc.
- Retweet columns and replies not needed.


**Tweets table**
- Erroneous datatype (id)

**Image prediction table**
- Erroneous datatype (tweet_id)
- Inconsistence dog breeds i.e., value separated with "_" instead of " " and also mixture of upper and lower case.

#### Tidiness
- There are 3 dataframe

**Twitter archive table**
- Dog stages in multiple columns (Instead of only one column).
- Timestamp column contains both time and date.

**Image prediction table**
- Only best image prediction is needed.

### Clean

In [24]:
twitter_archive_clean = twitter_archive.copy()
tweets_clean = tweets_use.copy()
image_pred_clean = image_pred.copy()

##### Data Quality 1
`Twitter archive`: - Erroneous data type

##### Define

Use `pandas.to_datetime()` function to convert `timestamp` column to datetime object and change the data type of `id` column to str.

##### Code

In [25]:
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

In [26]:
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)

##### Test

In [27]:
twitter_archive_clean['timestamp'].head()

0   2017-08-01 16:23:56+00:00
1   2017-08-01 00:17:27+00:00
2   2017-07-31 00:18:03+00:00
3   2017-07-30 15:58:51+00:00
4   2017-07-29 16:00:24+00:00
Name: timestamp, dtype: datetime64[ns, UTC]

##### Tidiness issue 1

Two values in timestamp column (date & time)

##### Define

Extract both time and date from the timestamp object and delete the timestamp column after extraction.

##### Code

In [28]:
twitter_archive_clean['date'] = pd.to_datetime(twitter_archive_clean['timestamp'].dt.date)
twitter_archive_clean['time'] = twitter_archive_clean['timestamp'].dt.time

# Removing timestamp from the dataframe
twitter_archive_clean.drop(columns=['timestamp'], inplace=True)

##### Test

In [29]:
twitter_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,date,time
0,892420643555336193,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,2017-08-01,16:23:56
1,892177421306343426,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,2017-08-01,00:17:27


In [30]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   source                      2356 non-null   object        
 4   text                        2356 non-null   object        
 5   retweeted_status_id         181 non-null    float64       
 6   retweeted_status_user_id    181 non-null    float64       
 7   retweeted_status_timestamp  181 non-null    object        
 8   expanded_urls               2297 non-null   object        
 9   rating_numerator            2356 non-null   int64         
 10  rating_denominator          2356 non-null   int64         
 11  name                        2356 non-null   object      

##### Data Quality 2

`Twitter archive`: **Misrepresentation of NA values**

##### Define

Use `dataframe.replace()` function to replace all the "None" strings in the dog_stages columns

##### Code

In [31]:
twitter_archive_clean.loc[:, "doggo":'puppo'] = twitter_archive_clean.loc[:, "doggo":'puppo'].replace({'None': np.nan})

##### Test

In [32]:
twitter_archive_clean.loc[:5, "doggo":'puppo']

,doggo,floofer,pupper,puppo
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN


In [33]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   source                      2356 non-null   object        
 4   text                        2356 non-null   object        
 5   retweeted_status_id         181 non-null    float64       
 6   retweeted_status_user_id    181 non-null    float64       
 7   retweeted_status_timestamp  181 non-null    object        
 8   expanded_urls               2297 non-null   object        
 9   rating_numerator            2356 non-null   int64         
 10  rating_denominator          2356 non-null   int64         
 11  name                        2356 non-null   object      

##### Tidiness issue 2

Twitter archive: **Dog stages requires only one column**

##### Define

Use melt() method to get all values of dog stages in one column and remove all redundant dog stage columns i.e., 'doggo', 'floofer', 'pupper', 'puppo'

In [34]:
twitter_archive_clean['dog_stage'] = twitter_archive_clean.melt(value_vars=('doggo', 'floofer', 'pupper', 'puppo'), var_name='dogs_col', value_name='dog_stage')['dog_stage']

In [35]:
twitter_archive_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

##### Test

In [36]:
# All columns in the dataframe
twitter_archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'source',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'date', 'time', 'dog_stage'],
      dtype='object')

In [37]:
twitter_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,date,time,dog_stage
0,892420643555336193,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,2017-08-01,16:23:56,NaN
1,892177421306343426,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,2017-08-01,00:17:27,NaN


In [38]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   source                      2356 non-null   object        
 4   text                        2356 non-null   object        
 5   retweeted_status_id         181 non-null    float64       
 6   retweeted_status_user_id    181 non-null    float64       
 7   retweeted_status_timestamp  181 non-null    object        
 8   expanded_urls               2297 non-null   object        
 9   rating_numerator            2356 non-null   int64         
 10  rating_denominator          2356 non-null   int64         
 11  name                        2356 non-null   object      

##### Data Quality 3

Redundant urls in a source column

##### Define

Use regular expression pattern to extract the text in between the \<a> tag

##### Code

In [39]:
twitter_archive_clean['source'] = twitter_archive_clean['source'].str.extract(r'<a.*>(.*)</a>')

##### Test

In [40]:
twitter_archive_clean['source'].head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

##### Data Quality 4

`Tweet archive`: **Unknow url at the end of tweet texts.**

##### Define

Remove the url at the end of each tweet text using `replace()` method in str

##### Code

In [41]:
twitter_archive_clean['text'] = twitter_archive_clean['text'].str.replace(r'\shttp\S+$', "", regex=True)

##### Test

In [42]:
twitter_archive_clean['text'].str.contains('http').sum()

29

##### Data Quality 5

`Twitter archive`: **Invalid dogs name**

##### Define

Remove all dogs name with lower case and replace all "None" with np.nan in the dataframe.

##### Code

In [43]:
twitter_archive_clean = twitter_archive_clean[~twitter_archive_clean.name.str.islower()]

twitter_archive_clean.replace({"None": np.nan}, inplace=True)

##### Test

In [44]:
twitter_archive_clean.name.value_counts()

Charlie       12
Lucy          11
Cooper        11
Oliver        11
Lola          10
              ..
Devón          1
Gert           1
Dex            1
Ace            1
Christoper     1
Name: name, Length: 931, dtype: int64

#### Data Quality 6

`Twitter archive`: **Invalid ratings representation**

##### Define

Extract both ratings numerator and denominator from the text column and use `astype()` method to change their type from str to float.

##### Code

In [45]:
twitter_archive_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_clean['text'].str.extract(r'(\d+(?:\.\d+)?)/(\d+)', expand=True)

In [46]:
twitter_archive_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_clean[['rating_numerator', 'rating_denominator']].astype(float)

##### Test

In [47]:
twitter_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,date,time,dog_stage
0,892420643555336193,NaN,NaN,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,Phineas,2017-08-01,16:23:56,NaN
1,892177421306343426,NaN,NaN,Twitter for iPhone,This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13.0,10.0,Tilly,2017-08-01,00:17:27,NaN


In [48]:
twitter_archive_clean['rating_numerator'].value_counts()

12.00      541
11.00      447
10.00      437
13.00      343
9.00       144
8.00        95
14.00       50
7.00        49
5.00        34
6.00        30
3.00        18
4.00        14
1.00         8
2.00         7
0.00         2
15.00        2
420.00       2
9.75         2
99.00        1
144.00       1
11.26        1
9.50         1
20.00        1
121.00       1
143.00       1
44.00        1
45.00        1
80.00        1
1776.00      1
50.00        1
165.00       1
13.50        1
17.00        1
11.27        1
666.00       1
24.00        1
182.00       1
84.00        1
960.00       1
88.00        1
Name: rating_numerator, dtype: int64

In [49]:
twitter_archive_clean['rating_denominator'].value_counts()

10.0     2227
11.0        3
20.0        2
50.0        2
80.0        2
0.0         1
15.0        1
70.0        1
7.0         1
150.0       1
90.0        1
40.0        1
130.0       1
110.0       1
16.0        1
120.0       1
Name: rating_denominator, dtype: int64

In [50]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2247 entries, 0 to 2355
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2247 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   source                      2247 non-null   object        
 4   text                        2247 non-null   object        
 5   retweeted_status_id         176 non-null    float64       
 6   retweeted_status_user_id    176 non-null    float64       
 7   retweeted_status_timestamp  176 non-null    object        
 8   expanded_urls               2188 non-null   object        
 9   rating_numerator            2247 non-null   float64       
 10  rating_denominator          2247 non-null   float64       
 11  name                        1502 non-null   object      

#### Data Quality 7

`Twitter archive`: **Huge ratings numerator and denominator are invalid**

##### Define

Use pandas `query()` method to extract dataframe with ratings denominator of 10 and ratings numerator <= 14. Evaluate the rating_numerator column by dividing it by 10. Make sure to remove ratings denominator and ratings numerator from the dataframe since we have the final rating.

##### Code

In [51]:
# Some ratings denominator > 10 are invalid for this analysis. 
twitter_archive_clean = twitter_archive_clean.query('rating_denominator == 10 & rating_numerator <= 14')

# final ratings in percentage
twitter_archive_clean['final_ratings'] = twitter_archive_clean['rating_numerator']*10

In [52]:
twitter_archive_clean.drop(columns=['rating_numerator', 'rating_denominator'], inplace=True)

##### Test

In [53]:
twitter_archive_clean['final_ratings'].head()

0    130.0
1    130.0
2    120.0
3    130.0
4    120.0
Name: final_ratings, dtype: float64

In [54]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2219 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2219 non-null   object        
 1   in_reply_to_status_id       68 non-null     float64       
 2   in_reply_to_user_id         68 non-null     float64       
 3   source                      2219 non-null   object        
 4   text                        2219 non-null   object        
 5   retweeted_status_id         174 non-null    float64       
 6   retweeted_status_user_id    174 non-null    float64       
 7   retweeted_status_timestamp  174 non-null    object        
 8   expanded_urls               2169 non-null   object        
 9   name                        1498 non-null   object        
 10  date                        2219 non-null   datetime64[ns]
 11  time                        2219 non-null   object      

#### Data Quality 8

`Twitter archive`: **Retweeted tweets and replies not needed.**

##### Define

Remove all retweeted tweets and replies

##### Code

In [55]:
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['retweeted_status_id'].isna()]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['in_reply_to_user_id'].isna()]

##### Test

In [56]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1977 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    1977 non-null   object        
 1   in_reply_to_status_id       0 non-null      float64       
 2   in_reply_to_user_id         0 non-null      float64       
 3   source                      1977 non-null   object        
 4   text                        1977 non-null   object        
 5   retweeted_status_id         0 non-null      float64       
 6   retweeted_status_user_id    0 non-null      float64       
 7   retweeted_status_timestamp  0 non-null      object        
 8   expanded_urls               1974 non-null   object        
 9   name                        1386 non-null   object        
 10  date                        1977 non-null   datetime64[ns]
 11  time                        1977 non-null   object      

#### Data Quality 9

`Twitter archive`: **Retweet columns and replies are no longer needed**

##### Define

Drop all retweet and replies columns using `drop()` method in pandas

##### Code

In [57]:
colto_drop = (i for i in twitter_archive_clean.columns if re.search(r'retweet.*|in_reply.*', i))

twitter_archive_clean.drop(columns=colto_drop, inplace=True)

##### Test

In [58]:
twitter_archive_clean.columns

Index(['tweet_id', 'source', 'text', 'expanded_urls', 'name', 'date', 'time',
       'dog_stage', 'final_ratings'],
      dtype='object')

In [59]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1977 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   tweet_id       1977 non-null   object        
 1   source         1977 non-null   object        
 2   text           1977 non-null   object        
 3   expanded_urls  1974 non-null   object        
 4   name           1386 non-null   object        
 5   date           1977 non-null   datetime64[ns]
 6   time           1977 non-null   object        
 7   dog_stage      80 non-null     object        
 8   final_ratings  1977 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 154.5+ KB


#### Data Quality 10

- `Image prediction`: **Erroneous datatype**
- `Tweets additional info`: **Erroneous datatype**

##### Define

Change the datatype of tweet_id column in both image prediction and tweets additional info table to string using astype() method in pandas

##### Code

In [60]:
image_pred_clean['tweet_id'] = image_pred_clean['tweet_id'].astype(str)
tweets_clean['id'] = tweets_clean['id'].astype(str)

##### Test

In [61]:
image_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [62]:
tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2354 non-null   object
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.3+ KB


#### Tidiness issue 3

`Image prediction`: **Only best image prediction is needed**

##### Define

Take the best image prediction for the images in the tweets. It will be reasonable to drop all other columns except tweet id since best prediction has been taken.

In [63]:
def best_prediction(tweet_image):
    """
        This function will return the best image prediction
    """
    pred = None
    if (tweet_image['p1_conf'] >= tweet_image['p2_conf']
             and
        tweet_image['p1_conf'] >= tweet_image['p3_conf']):
        pred = tweet_image['p1']
    elif tweet_image['p2_conf'] > tweet_image['p3_conf']:
        pred = tweet_image['p2']
    else:
        pred = tweet_image['p3'] 
    return pred


In [64]:
image_pred_clean['dog_breed'] = image_pred_clean.apply(best_prediction, axis=1)

In [65]:
image_pred_clean = image_pred_clean[['tweet_id', 'dog_breed']]

#### Data Quality 11

`Image prediction`: **Inconsistence dog breeds**

##### Define

Replace "_" with " " and capitalize all dog breeds.

##### Code

In [66]:
# replacing all underscore with space
image_pred_clean['dog_breed'] = image_pred_clean['dog_breed'].str.replace("_", " ")

In [67]:
# capitalizing all dogs breed
image_pred_clean['dog_breed'] = image_pred_clean['dog_breed'].str.capitalize()

##### Test

In [68]:
image_pred_clean.head()

,tweet_id,dog_breed
0,666020888022790149,Welsh springer spaniel
1,666029285002620928,Redbone
2,666033412701032449,German shepherd
3,666044226329800704,Rhodesian ridgeback
4,666049248165822465,Miniature pinscher


In [69]:
image_pred_clean.tail()

,tweet_id,dog_breed
2070,891327558926688256,Basset
2071,891689557279858688,Paper towel
2072,891815181378084864,Chihuahua
2073,892177421306343426,Chihuahua
2074,892420643555336193,Orange


In [70]:
image_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   2075 non-null   object
 1   dog_breed  2075 non-null   object
dtypes: object(2)
memory usage: 32.5+ KB


#### Tidiness issue 4

There are 3 Dataframe which suppose to be 1 because they are all referring to tweets details

##### Define

Merge the three dataframes together base on their tweet id

##### Code

In [71]:
df_clean = pd.merge(twitter_archive_clean, image_pred_clean, on='tweet_id')
df_clean = df_clean.merge(tweets_clean, left_on= 'tweet_id', right_on='id').drop('id', axis=1)

##### Test

In [72]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1857 entries, 0 to 1856
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tweet_id        1857 non-null   object        
 1   source          1857 non-null   object        
 2   text            1857 non-null   object        
 3   expanded_urls   1857 non-null   object        
 4   name            1345 non-null   object        
 5   date            1857 non-null   datetime64[ns]
 6   time            1857 non-null   object        
 7   dog_stage       70 non-null     object        
 8   final_ratings   1857 non-null   float64       
 9   dog_breed       1857 non-null   object        
 10  retweet_count   1857 non-null   int64         
 11  favorite_count  1857 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 188.6+ KB
